In [6]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS
chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/ chmod +x /tmp/chromedriver mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [999 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,298 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease


In [2]:
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [9]:
!pip install chromedriver-autoinstaller
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [14]:
import chromedriver_autoinstaller  # setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
# ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/128/chromedriver'

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

In [15]:
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.scrapethissite.com/pages/ajax-javascript/')

years = driver.find_elements(By.CLASS_NAME, 'year-link')
films = []
delay = 3

for year in years:
  year.click()
  time.sleep(3)
  soup = BeautifulSoup(driver.page_source)
  films.extend(soup.find_all(class_="film"))

driver.quit()

In [16]:
film_title = []
film_nominations = []
film_awards = []
film_best_picture = []

for film in films:
  film_title.append(film.find(class_="film-title").text.strip())
  film_nominations.append(film.find(class_="film-nominations").text.strip())
  film_awards.append(film.find(class_="film-awards").text.strip())
  film_best_picture.append(1 if film.find(class_='glyphicon glyphicon-flag') else 0)

In [17]:
FinalData = pd.DataFrame(
    {'film_title': film_title,
     'film_nominations':film_nominations,
     'film_awards':film_awards,
     'film_best_picture':film_best_picture,
    })

FinalData

,film_title,film_nominations,film_awards,film_best_picture
0,Spotlight,6,2,1
1,Mad Max: Fury Road,10,6,0
2,The Revenant,12,3,0
3,Bridge of Spies,6,1,0
4,The Big Short,5,1,0
...,...,...,...,...
82,The Lost Thing,1,1,0
83,God of Love,1,1,0
84,The Wolfman,1,1,0
85,Strangers No More,1,1,0


In [18]:
FinalData.to_csv('FinalData.csv')